In [1]:
# !pip install transformers

In [2]:
import os, gc, sys, time, collections, random
import numpy as np
import pandas as pd

from typing import Dict, Optional, Union, Any, List, Tuple

from pathlib import Path
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn

import torch.utils.data as D
from torch.utils.data.dataset import Dataset, IterableDataset
from torch.utils.data.dataloader import DataLoader

from transformers import DistilBertTokenizerFast
from transformers import DistilBertModel
from transformers import BertTokenizerFast
from transformers import BertModel
from transformers import Trainer
from transformers import AutoModel, AutoTokenizer
from transformers.data.processors.utils import InputFeatures
from transformers import TrainingArguments
from transformers.trainer_utils import EvalLoopOutput
from transformers.trainer import logging
from transformers.file_utils import is_torch_tpu_available, is_sagemaker_mp_enabled
from transformers.trainer_pt_utils import find_batch_size, nested_concat, nested_numpify, nested_truncate, nested_detach

### Folders and Dataframes

In [3]:
DATA_PATH = Path('/home/commonlit/data/')
assert DATA_PATH.exists()
MODELS_PATH = Path('/home/commonlit/models/')
if not MODELS_PATH.exists():
    os.mkdir(MODELS_PATH)
assert MODELS_PATH.exists()

In [4]:
!ls {DATA_PATH}

commonlitreadabilityprize.zip  sample_submission.csv  test.csv	train.csv


In [5]:
train_df = pd.read_csv(DATA_PATH/'train.csv')
test_df = pd.read_csv(DATA_PATH/'test.csv')
sample_df = pd.read_csv(DATA_PATH/'sample_submission.csv')

In [6]:
train_df

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845
...,...,...,...,...,...,...
2829,25ca8f498,https://sites.ehe.osu.edu/beyondpenguins/files...,CC BY-SA 3.0,When you think of dinosaurs and where they liv...,1.711390,0.646900
2830,2c26db523,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,So what is a solid? Solids are usually hard be...,0.189476,0.535648
2831,cd19e2350,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,The second state of matter we will discuss is ...,0.255209,0.483866
2832,15e2e9e7a,https://en.wikibooks.org/wiki/Geometry_for_Ele...,CC BY-SA 3.0,Solids are shapes that you can actually touch....,-0.215279,0.514128


### Prepare Cross Validation

In [7]:
target = train_df['target'].to_numpy()

In [8]:
num_bins = int(np.floor(np.log2(len(train_df))) + 1)
train_df.loc[:, 'bins'] = pd.cut(train_df['target'], bins=num_bins, labels=False)

In [9]:
train_df[['target', 'bins']].groupby(['bins']).agg(['mean', 'count'])

target      
          mean count
bins                
0    -3.413097    43
1    -2.969369    79
2    -2.526589   172
3    -2.106393   269
4    -1.652726   366
5    -1.201502   418
6    -0.748738   481
7    -0.309800   405
8     0.130016   312
9     0.560407   183
10    0.978923    83
11    1.399764    23

In [10]:
kf = StratifiedKFold(n_splits=num_bins)

In [11]:
for i, (t_, v_) in enumerate(kf.split(X=train_df, y=train_df.bins.values)):
    train_df.loc[v_, 'kfold'] = i

In [12]:
train_df['kfold'] = train_df['kfold'].astype(np.uint8)

In [13]:
train_df = train_df.drop('bins', axis=1)

In [14]:
train_df

,id,url_legal,license,excerpt,target,standard_error,kfold
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009,0
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805,0
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676,0
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007,0
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845,0
...,...,...,...,...,...,...,...
2829,25ca8f498,https://sites.ehe.osu.edu/beyondpenguins/files...,CC BY-SA 3.0,When you think of dinosaurs and where they liv...,1.711390,0.646900,11
2830,2c26db523,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,So what is a solid? Solids are usually hard be...,0.189476,0.535648,11
2831,cd19e2350,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,The second state of matter we will discuss is ...,0.255209,0.483866,11
2832,15e2e9e7a,https://en.wikibooks.org/wiki/Geometry_for_Ele...,CC BY-SA 3.0,Solids are shapes that you can actually touch....,-0.215279,0.514128,11


In [15]:
bin_list = list(range(num_bins))
random.shuffle(bin_list)
bin_list

[10, 3, 6, 7, 9, 0, 1, 2, 8, 11, 5, 4]

### Metrics

In [16]:
def rmse_score(y_true, y_pred):
    return np.sqrt(np.mean((y_pred - y_true) ** 2))

def rmse_score_2(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [17]:
a = np.random.rand(10)
b = np.random.rand(10)

In [18]:
rmse_score(a, b), rmse_score_2(a, b)

(0.43627309883301485, 0.43627309883301485)

### Configuration

In [19]:
class CONFIG():
    model_name = 'bert'
    batch_size = 80
    max_len = 256
    save_dir = f'trained/{model_name}'
    num_workers = 2
    epochs = 20
    pretrained_transformers_model = f'{model_name}-base-uncased'
    split = 2

In [20]:
cfg = CONFIG()

### Prepare train test split

In [21]:
def create_split(fold = [1]):
    valid_df = train_df[train_df['kfold'].isin(fold)]
    valid_text = valid_df['excerpt'].values
    valid_target = valid_df['target'].values
    training_df = train_df[~train_df['kfold'].isin(fold)]
    train_text = training_df['excerpt'].values
    train_target = training_df['target'].values
    return train_text, train_target, valid_text, valid_target

In [22]:
train_text, train_target, valid_text, valid_target = create_split([cfg.split])
len(train_text), len(valid_text)

(2598, 236)

### Prepare Tokenizers

In [23]:
tokenizer = AutoTokenizer.from_pretrained(cfg.pretrained_transformers_model)
# Save the tokenizer so that you can download the files and move it to a Kaggle dataset.
tokenizer.save_pretrained(cfg.save_dir)

('trained/bert/tokenizer_config.json',
 'trained/bert/special_tokens_map.json',
 'trained/bert/vocab.txt',
 'trained/bert/added_tokens.json',
 'trained/bert/tokenizer.json')

In [24]:
encoded_dict = tokenizer(train_df['excerpt'].values[0],
                                return_tensors='pt',
                                max_length=cfg.max_len,
                                padding='max_length',
                                truncation=True)
decoded = tokenizer.decode(encoded_dict["input_ids"].squeeze())
decoded

'[CLS] when the young people returned to the ballroom, it presented a decidedly changed appearance. instead of an interior scene, it was a winter landscape. the floor was covered with snow - white canvas, not laid on smoothly, but rumpled over bumps and hillocks, like a real snow field. the numerous palms and evergreens that had decorated the room, were powdered with flour and strewn with tufts of cotton, like snow. also diamond dust had been lightly sprinkled on them, and glittering crystal icicles hung from the branches. at each end of the room, on the wall, hung a beautiful bear - skin rug. these rugs were for prizes, one for the girls and one for the boys. and this was the game. the girls were gathered at one end of the room and the boys at the other, and one end was called the north pole, and the other the south pole. each player was given a small flag which they were to plant on reaching the pole. this would have been an easy matter, but each traveller was obliged to wear snowsho

In [25]:
encoded_dict['input_ids'].shape

torch.Size([1, 256])

In [26]:
def convert_to_list(t):
    return t.flatten().long()

class CommonLitDataset(nn.Module):
    def __init__(self, text, target, tokenizer, max_len=128):
        self.excerpt = text
        self.target = target
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],
                                return_tensors='pt',
                                max_length=self.max_len,
                                padding='max_length',
                                truncation=True)
        return InputFeatures(input_ids=convert_to_list(encode['input_ids']),
                      attention_mask=convert_to_list(encode['attention_mask']),
                      label=torch.tensor(self.target[idx]))
    
    def __len__(self):
        return len(self.excerpt)

In [27]:
def create_train_valid_ds(tokenizer, train_text, train_target, valid_text, valid_target):
    train_ds = CommonLitDataset(train_text, train_target, tokenizer, cfg.max_len)
    valid_ds = CommonLitDataset(valid_text, valid_target, tokenizer, cfg.max_len)
    return train_ds, valid_ds

In [28]:
# train_dl = D.DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=True, num_workers=cfg.num_workers)
# train_dl = D.DataLoader(valid_ds, batch_size=cfg.batch_size, shuffle=False, num_workers=cfg.num_workers)

In [29]:
# encode, target = next(iter(train_dl))

In [30]:
# encode.keys(), target.shape, encode['input_ids'].shape, encode['attention_mask'].shape

In [31]:
# encode['input_ids'][0].squeeze()

### Model

In [32]:
# You can use a Transformer model of your choice.
# transformer_model = DistilBertModel.from_pretrained(cfg.pretrained_transformers_model)
transformer_model = AutoModel.from_pretrained(cfg.pretrained_transformers_model)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [33]:
# transformer_out = transformer_model(input_ids=encode['input_ids'].squeeze(), attention_mask=encode['attention_mask'].squeeze())

In [34]:
# dict(transformer_out)['last_hidden_state'].shape

In [35]:
# torch.mean(transformer_out.last_hidden_state, axis=1).shape

In [36]:
# sample_layer = nn.Linear(768, 1)

In [37]:
# sample_layer(torch.mean(transformer_out.last_hidden_state, axis=1)).shape

In [38]:
from transformers import PreTrainedModel

class CommonLitModel(PreTrainedModel):
    def __init__(self):
        super(PreTrainedModel, self).__init__()
        self.transformer_model = AutoModel.from_pretrained(cfg.pretrained_transformers_model)
        self.drop = nn.Dropout(0.5)
        self.out = nn.Linear(768, 1)
        self.config = self.transformer_model.config
        
    def forward(self, input_ids, attention_mask):
        transformer_out = self.transformer_model(input_ids=input_ids.squeeze(), attention_mask=attention_mask.squeeze(), output_hidden_states=False)
        x = transformer_out.pooler_output
#         x = transformer_out.last_hidden_state[:, 0, :]
        x = self.drop(x)
        x = self.out(x)
        return x
    
    def floating_point_ops(self, inputs: Dict[str, Union[torch.Tensor, Any]]):
        """
        For models that inherit from :class:`~transformers.PreTrainedModel`, uses that method to compute the number of
        floating point operations for every backward + forward pass. If using another model, either implement such a
        method in the model or subclass and override this method.
        Args:
            inputs (:obj:`Dict[str, Union[torch.Tensor, Any]]`):
                The inputs and targets of the model.
        Returns:
            :obj:`int`: The number of floating-point operations.
        """
        return 0

In [39]:
model = CommonLitModel()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [40]:
encoded_dict.input_ids.shape

torch.Size([1, 256])

In [41]:
transformer_model = transformer_model.cuda()
sample_out = transformer_model(encoded_dict.input_ids.cuda(), encoded_dict.attention_mask.cuda(), output_hidden_states=True)

In [42]:
train_ds, valid_ds = create_train_valid_ds(tokenizer, train_text, train_target, valid_text, valid_target)

In [43]:
encode = train_ds[0]

In [44]:
encode.attention_mask.unsqueeze(0).shape, encoded_dict.input_ids.shape

(torch.Size([1, 256]), torch.Size([1, 256]))

In [45]:
sample_out = transformer_model(encode.input_ids.unsqueeze(0).cuda(), encode.attention_mask.unsqueeze(0).cuda())

In [46]:
sample_out.pooler_output.shape

torch.Size([1, 768])

### Training

In [47]:
import wandb
wandb.init(project="commonlit")

wandb: Currently logged in as: gilf (use `wandb login --relogin` to force relogin)


In [48]:
loss_fct = nn.MSELoss()

In [49]:
def create_training_args(fold):
    training_args = TrainingArguments(
        output_dir=str(MODELS_PATH/f'{cfg.model_name}-{fold}'),
        overwrite_output_dir=True,
        do_train=True,
        do_eval=True,
        per_device_train_batch_size=cfg.batch_size,
        per_device_eval_batch_size=cfg.batch_size,
        num_train_epochs=cfg.epochs,
        logging_strategy="epoch",
        logging_first_step=True,
        save_steps=40000,
        fp16=True,
        evaluation_strategy="epoch",
        save_total_limit = 3,
        load_best_model_at_end=True,
        metric_for_best_model='mse',
        greater_is_better=False,
        gradient_accumulation_steps=2,
        learning_rate=5e-5
    )
    return training_args

In [50]:
training_args = create_training_args(cfg.split)

In [51]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    return {'mse': mean_squared_error(logits, labels), 'rmse': rmse_score_2(logits, labels)}

In [52]:
tokenizer = AutoTokenizer.from_pretrained(cfg.pretrained_transformers_model)

In [53]:
logger = logging.get_logger(__name__)

class CommonLitTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        input_ids = inputs.pop("input_ids")
        attention_mask = inputs.pop("attention_mask")
        outputs = model(input_ids, attention_mask)
        logits = outputs
        loss = loss_fct(logits.flatten(),
                        labels.float().flatten())
        zero_cat = torch.zeros([1, 1]).to(outputs.device)
        return (loss, torch.cat([zero_cat, outputs])) if return_outputs else loss

In [54]:
!rm -rf /home/commonlit/models/bert-*

In [55]:
%%time

from transformers import EarlyStoppingCallback

bin_step = 1
bestmodels = []
eval_rmses = []
for i in range(0, num_bins, bin_step):
    wandb.init()
    train_bins = bin_list[i:i+bin_step]
    print('train_bins', f'{i}: {train_bins}')
    tokenizer = AutoTokenizer.from_pretrained(cfg.pretrained_transformers_model)
    train_text, train_target, valid_text, valid_target = create_split([i])
    train_ds, valid_ds = create_train_valid_ds(tokenizer, train_text, train_target, valid_text, valid_target)
    training_args = create_training_args(i)
    model = CommonLitModel()
    trainer = CommonLitTrainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=valid_ds,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=9)]
    )
    trainer.train()
    trainer.save_model()
    print('training_args.output_dir', training_args.output_dir)
    tokenizer.save_pretrained(training_args.output_dir)
    result = trainer.evaluate()
    bestmodels.append(trainer.state.best_model_checkpoint)
    print('best_model_checkpoint', trainer.state.best_model_checkpoint)
    print('result', result)
    eval_rmses.append(result['eval_rmse'])

train_bins 0: [10]


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch,Training Loss,Validation Loss,Mse,Rmse
0,1.288900,0.597303,0.597303,0.772854
1,0.500100,0.502688,0.502688,0.709005
2,0.318100,0.307054,0.307054,0.554124
3,0.223000,0.493393,0.493393,0.702419
4,0.151800,0.410280,0.410280,0.640531
5,0.104400,0.583482,0.583481,0.763860
6,0.124800,0.646072,0.646072,0.803786
7,0.085900,0.371755,0.371755,0.609718
8,0.055500,0.408381,0.408382,0.639047
9,0.047400,0.461010,0.461010,0.678977


training_args.output_dir /home/commonlit/models/bert-0


best_model_checkpoint /home/commonlit/models/bert-0/checkpoint-48
result {'eval_loss': 0.307053804397583, 'eval_mse': 0.3070538341999054, 'eval_rmse': 0.5541244149208069, 'eval_runtime': 1.3412, 'eval_samples_per_second': 176.708, 'epoch': 11.97, 'eval_mem_cpu_alloc_delta': 0, 'eval_mem_gpu_alloc_delta': 0, 'eval_mem_cpu_peaked_delta': 0, 'eval_mem_gpu_peaked_delta': 944294400}


...

train_bins 11: [4]


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch,Training Loss,Validation Loss,Mse,Rmse
0,0.986600,0.485803,0.485803,0.696996
1,0.445700,0.468408,0.468408,0.684403
2,0.324300,0.348973,0.348973,0.590740
3,0.232300,0.379724,0.379724,0.616217
4,0.167500,0.372726,0.372726,0.610513
5,0.132900,0.467292,0.467292,0.683588
6,0.093100,0.359733,0.359733,0.599778
7,0.097400,0.582988,0.582988,0.763537
8,0.074500,0.371522,0.371522,0.609526
9,0.059000,0.569263,0.569263,0.754495


training_args.output_dir /home/commonlit/models/bert-11


best_model_checkpoint /home/commonlit/models/bert-11/checkpoint-48
result {'eval_loss': 0.3489733636379242, 'eval_mse': 0.3489733636379242, 'eval_rmse': 0.5907396674156189, 'eval_runtime': 1.3419, 'eval_samples_per_second': 175.869, 'epoch': 11.97, 'eval_mem_cpu_alloc_delta': 0, 'eval_mem_gpu_alloc_delta': 0, 'eval_mem_cpu_peaked_delta': 0, 'eval_mem_gpu_peaked_delta': 944294400}
CPU times: user 1h 43min 11s, sys: 22min 13s, total: 2h 5min 25s
Wall time: 1h 11min 2s


In [56]:
trainer.train()

Epoch,Training Loss,Validation Loss,Mse,Rmse
0,0.225100,0.385473,0.385473,0.620865
1,0.192200,0.463153,0.463153,0.680553
2,0.168400,0.533111,0.533111,0.730144
3,0.132500,0.398889,0.398889,0.631576
4,0.113000,0.527647,0.527647,0.726393
5,0.099700,0.486656,0.486656,0.697608
6,0.087900,0.504787,0.504787,0.710484
7,0.078600,0.470719,0.470719,0.686090
8,0.081100,0.471283,0.471283,0.686501
9,0.076200,0.471283,0.471283,0.686501


TrainOutput(global_step=160, training_loss=0.12613386586308478, metrics={'train_runtime': 225.1119, 'train_samples_per_second': 1.422, 'total_flos': 0, 'epoch': 9.97, 'train_mem_cpu_alloc_delta': 278528, 'train_mem_gpu_alloc_delta': 512, 'train_mem_cpu_peaked_delta': 187420672, 'train_mem_gpu_peaked_delta': 18555851264})

In [57]:
BEST_MODEL_FOLDER = MODELS_PATH/cfg.model_name/'best'
!rm -rf {BEST_MODEL_FOLDER}
!mkdir -p {BEST_MODEL_FOLDER}

In [58]:
from shutil import copyfile

for i, best_model in enumerate(bestmodels):
    best_model_file = f'{best_model}/pytorch_model.bin'
    if Path(best_model_file).exists():
        copyfile(best_model_file, f'{BEST_MODEL_FOLDER}/{i}_pytorch_model.bin')
        tokenizer_path = Path(BEST_MODEL_FOLDER/f'tokenizer-{i}')
        tokenizer_path.mkdir(parents=True, exist_ok=True)
        assert tokenizer_path.exists()

        tokenizer_json = Path(f'{MODELS_PATH/cfg.model_name}-{i}/tokenizer.json')
        assert tokenizer_json.exists()
        copyfile(tokenizer_json, tokenizer_path/'tokenizer.json')

        vocab_txt = Path(f'{MODELS_PATH/cfg.model_name}-{i}/vocab.txt')
        assert vocab_txt.exists()
        copyfile(vocab_txt, tokenizer_path/'vocab.txt')

        config_json = Path(f'{MODELS_PATH/cfg.model_name}-{i}/config.json')
        assert config_json.exists()
        copyfile(config_json, tokenizer_path/'config.json')
    else:
        print(f'{best_model_file} is missing')

/home/commonlit/models/bert-11/checkpoint-48/pytorch_model.bin is missing


In [59]:
bestmodels

['/home/commonlit/models/bert-0/checkpoint-48',
 '/home/commonlit/models/bert-1/checkpoint-192',
 '/home/commonlit/models/bert-2/checkpoint-96',
 '/home/commonlit/models/bert-3/checkpoint-48',
 '/home/commonlit/models/bert-4/checkpoint-64',
 '/home/commonlit/models/bert-5/checkpoint-128',
 '/home/commonlit/models/bert-6/checkpoint-80',
 '/home/commonlit/models/bert-7/checkpoint-208',
 '/home/commonlit/models/bert-8/checkpoint-64',
 '/home/commonlit/models/bert-9/checkpoint-144',
 '/home/commonlit/models/bert-10/checkpoint-96',
 '/home/commonlit/models/bert-11/checkpoint-48']

In [60]:
import shutil

shutil.make_archive(MODELS_PATH/cfg.model_name/'best_models', 'zip', BEST_MODEL_FOLDER)

'/home/commonlit/models/bert/best_models.zip'